# **Dowload Data from Kaggle**

In [ ]:
 ! pip install -q kaggle

In [ ]:
%cd /content/drive/MyDrive/My_works/HAUI-TriTueNhanTao/ClassificationPneumonia

In [ ]:
 ! mkdir kaggle

In [ ]:
! cp kaggle.json kaggle/

In [ ]:
! chmod 600 kaggle/kaggle.json

In [ ]:
! kaggle datasets list

In [ ]:
! kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

In [ ]:
! mkdir datasets

In [ ]:
! unzip chest-xray-pneumonia.zip -d datasets

# **Import Library**

In [ ]:
!pip install tensorflow

In [ ]:
import numpy as np 
import pandas as pd 
from numpy import *
import os
import matplotlib.pyplot as plt 
from keras.models import Sequential
from tensorflow.keras import optimizers
import tensorflow as tf
from keras.layers import Conv2D, BatchNormalization, Activation, MaxPooling2D, Dropout, Dense, Flatten
from PIL import Image 
from keras.preprocessing.image import ImageDataGenerator 
from keras import layers, models, optimizers 

# **Path to data**

In [ ]:
base_dir = '/content/drive/MyDrive/My_works/HAUI-TriTueNhanTao/ClassificationPneumonia/data/chest_xray'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

In [ ]:
from google.colab import drive

drive.mount('/content/drive/')

In [ ]:
imagen = plt.imread(os.path.join(train_dir, 'NORMAL/IM-0117-0001.jpeg'))
plt.imshow(imagen)

# **Loading images from directory using generator**

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest'
                                   )
# val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=20,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=20,
    class_mode='binary'
)

# val_generator = val_datagen.flow_from_directory(
#     val_dir,
#     target_size=(224, 224),
#     batch_size=4,
#     class_mode='binary'
# )

# **Build Model CNN**

In [ ]:
def create_model(input_shape , num_class):
  model = Sequential();
  model.add(Conv2D(32, (3, 3), activation='relu', input_shape= input_shape)) 
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, (3, 3), activation='relu')) 
  model.add(MaxPooling2D((2, 2))) 
  model.add(Conv2D(128, (3, 3), activation='relu')) 
  model.add(MaxPooling2D((2, 2))) 
  model.add(Conv2D(128, (3, 3), activation='relu')) 
  model.add(MaxPooling2D((2, 2))) 
  model.add(Flatten())
  model.add(Dropout(0.2))
  model.add(Dense(512, activation='relu')) 
  model.add(Dense(num_class, activation='softmax'))

  return model 

In [ ]:
input_shape = (224, 224, 3)
num_class = 2

In [ ]:
model = create_model(input_shape, num_class)
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
history = model.fit_generator(train_generator,
                              steps_per_epoch=30,
                              epochs=100,
                              # validation_data=val_generator,
                              # validation_steps=4
                              )

# **Evaluate**

In [ ]:
test_loss, test_acc = model.evaluate_generator(test_generator, steps=30) 
print('test acc:', test_acc)

# **Save and Load Model**

In [ ]:
model.save("model/model2.10.h5")

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/My_works/HAUI-TriTueNhanTao/ClassificationPneumonia/model/model2.10.h5')

# **Predict**

In [ ]:
from google.colab.patches import cv2_imshow
import cv2

In [ ]:
from google.colab import files
files.upload()

In [ ]:
image_path = '/content/drive/MyDrive/My_works/HAUI-TriTueNhanTao/ClassificationPneumonia/Normal.jpg'
img = cv2.imread(image_path)
cv2_imshow(img)

In [ ]:
image_path = '/content/drive/MyDrive/My_works/HAUI-TriTueNhanTao/ClassificationPneumonia/Normal.jpg'
image = array(Image.open(image_path).resize((224,224)))
pred = model.predict(image.reshape(1, 224 , 224, 3))
print('Có bị bệnh viên phổi' if np.argmax(pred) == 1 else 'Không bị bệnh viêm phổi ')